In [1]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

In [2]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [3]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)



In [4]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [5]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [6]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [7]:
nj_current_assignment = pd.read_csv('/content/New Jersey Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

,GEOID20,District
0,34033020001,2
1,34033001501,2
2,34033042009,2
3,34033000703,2
4,34033042001,2
...,...,...
6356,34039045302,7
6357,34039045303,7
6358,34039045404,7
6359,34039045801,7


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [8]:
nj_precinct_data = pd.read_csv('/content/New Jersey Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [9]:
shpfile = '/content/New Jersey Data/nj_vtd_2020_bound.shp'
dbffile = '/content/New Jersey Data/nj_vtd_2020_bound.dbf'
shxfile = '/content/New Jersey Data/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom


ShapefileException: ignored

### This is the precinct boundary data

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv.

In [10]:
nj_contiguity = pd.read_csv('/content/New Jersey Data/Contiguity_nj.csv', header=None)
nj_contiguity


,0,1
0,34033020001,"['34033001501', '34033060E01', '34033020002', ..."
1,34033001501,"['34033020001', '34033042009', '34033060E01', ..."
2,34033042009,"['34033001501', '34033030001', '34033004204', ..."
3,34033000703,"['34033042001', '34033007001', '34033030001', ..."
4,34033042001,"['34033000703', '34033007001', '34033030001', ..."
...,...,...
6356,34039045302,"['34039045403', '34039045402', '34039045101', ..."
6357,34039045303,"['34039045601', '34039045403', '34039045603', ..."
6358,34039045404,"['34039045804', '34039045601', '34039045403', ..."
6359,34039045801,"['34039020207', '34039045804', '34039020206', ..."


In [11]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(12, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [12]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))


KeyError: ignored

In [13]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))
print(sum(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12)))

{1: 775340, 2: 778354, 3: 778489, 4: 767834, 5: 774454, 6: 778516, 7: 785173, 8: 800074, 9: 766863, 10: 746178, 11: 769523, 12: 768196}
78


# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West.
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [ ]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError:
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

<ipython-input-117-97b5395461a6>:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
<ipython-input-117-97b5395461a6>:7: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':


In [ ]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))


In [ ]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

In [1]:
import pandas as pd
nj_unfair_assignment = nj_current_assignment.copy()


nj_contiguity.rename(columns={'Precinct': 'GEOID20', 1: 'neighbors'}, inplace=True)
precincts_add = 0


#UNFAIR MAP CODE
#choosing to make district 3 unfair because of current split between democrats and republicans
#this method will add precincts on the border of district 3 if they have a higher democratic population
#Precinct added:  2574 changed so that this map is at least 10 percent different than current map

#only flipped from neighors so that contiguity is never broken
while precincts_add < 2500:

    merged_df = pd.merge(nj_contiguity, nj_current_assignment, on='GEOID20')
    unfair_district = 3

    district_seven_precincts = merged_df[merged_df['District'] == unfair_district]

    district_seven_neighbors = district_seven_precincts['Neighbors'].tolist()
    nested_list_of_lists = [ast.literal_eval(geoid_str) for geoid_str in district_seven_neighbors]
    flat_district_seven_neighbors = [geoid for sublist in nested_list_of_lists for geoid in sublist]


    neighbors_df = pd.DataFrame(columns=['Neighbor', 'District'])

    for neighbor in flat_district_seven_neighbors:

      if neighbor in nj_current_assignment['GEOID20'].values:
          district = nj_current_assignment[nj_current_assignment['GEOID20'] == neighbor]['District'].values[0]
          neighbors_df = neighbors_df.append({'Neighbor': neighbor, 'District': district}, ignore_index=True)


    neighbors_not_in_district_seven = neighbors_df[neighbors_df['District'] != 3]
    neighbors_not_in_district_seven = neighbors_not_in_district_seven.rename(columns={'Neighbor': 'GEOID20'})



    merge = pd.merge(neighbors_not_in_district_seven, nj_precinct_data, on = "GEOID20")
    for index, row in merge.iterrows():
        precinct = row['GEOID20']
        democrats_total = row["Dem_2020_Pres"]
        republican_total = row["Rep_2020_Pres"]
        if democrats_total >= republican_total:
            old_district = nj_unfair_assignment.loc[nj_unfair_assignment["GEOID20"] == precinct, "District"]
            nj_unfair_assignment.loc[nj_unfair_assignment["GEOID20"] == precinct, "District"] = 3
            precincts_add += 1

#Precinct added:  2574

print("Precinct added: " , precincts_add)

filtered_rows = nj_precinct_data[(nj_precinct_data['District'] == 1) &
                                 ((nj_precinct_data['White_2020_Total'] == 771) |
                                  (nj_precinct_data['White_2020_Total'] == 619) |
                                  (nj_precinct_data['Black_2020_Total'] == 419))]
nj_unfair_assignment.loc[nj_unfair_assignment["GEOID20"] == 34005140006, "District"] = 3
nj_unfair_assignment.loc[nj_unfair_assignment["GEOID20"] == 34005140007, "District"] = 3
nj_unfair_assignment.loc[nj_unfair_assignment["GEOID20"] == 34005140008, "District"] = 3
precincts_to_reassign = ['34005140006', '34005140007', '34005140008']
nj_unfair_assignment.loc[nj_unfair_assignment['GEOID20'].isin(precincts_to_reassign), 'District'] = 3


columns_to_keep = ['GEOID20', 'District']
nj_unfair_assignment = nj_unfair_assignment[columns_to_keep]
nj_unfair_assignment.to_csv("maps/redistricting_unfair_assignment.csv", index=False)

NameError: ignored

In [19]:
import numpy as np
import pandas as pd
import ast
#flipstep:

#1. make 12 lists of each districts precincts
#2. make 12 lists of precinct neighbors of each district
#3. need to flip 650 precincts all of which have around the same population (between 200 and 500)
#4. for each district need to flip 100 precincts
#5. go through the neighbors of the precinct and randomly select 5 and flip to the current district
#this code accounts for neigbors in same district
nj_new_assignment = nj_current_assignment.copy()
nj_contiguity.rename(columns={'Precinct': 'GEOID20', 1: 'Neighbors'}, inplace=True)

merged_df = pd.merge(nj_contiguity, nj_current_assignment, on='GEOID20')

num_of_flips = 0
while num_of_flips < 650:
  for i in range(1, 13):
      print(f"Processing District {i}")

      district_precincts = merged_df[merged_df['District'] == i]
      district_neighbors = [ast.literal_eval(geoid_str) for geoid_str in district_precincts['Neighbors']]
      district_neighbors = [geoid for sublist in district_neighbors for geoid in sublist if len(nj_current_assignment.loc[nj_current_assignment['GEOID20'] == geoid, 'District'].values) > 0 and nj_current_assignment.loc[nj_current_assignment['GEOID20'] == geoid, 'District'].values[0] != i]

      print(len(district_neighbors))

      if len(district_neighbors) < 100:
          #print(f"Not enough neighbors for District {i}. Skipping.")
          continue

      flipped_precincts = np.random.choice(district_neighbors, 50)

      for flip in flipped_precincts:
          old_assignment_district = nj_current_assignment.loc[nj_current_assignment['GEOID20'] == flip, 'District'].values[0]
          print(f"Flipping {flip} from District {old_assignment_district} to District {i}")

          nj_new_assignment.loc[nj_new_assignment['GEOID20'] == flip, 'District'] = i
          num_of_flips += 1
          for j in range(1,13):
            if(isDistrictContiguous(j,nj_new_assignment,nj_contiguity) is False):
              nj_new_assignment.loc[nj_new_assignment['GEOID20']==flip,'District']= old_assignment_district
              num_of_flips -= 1
              break
          print(f"Total number of flips: {num_of_flips}")
          if num_of_flips > 650:
              break

print(f"Total number of flips: {num_of_flips}")

nj_new_assignment.to_csv("Recitation maps/new_jersey_fair.csv", index=False)



Processing District 1
117
Flipping 34005065020 from District 3 to District 1
Total number of flips: 0
Flipping 34005040004 from District 3 to District 1
Total number of flips: 1
Flipping 34015005006 from District 2 to District 1
Total number of flips: 2
Flipping 34005125006 from District 3 to District 1
Total number of flips: 3
Flipping 34005040007 from District 3 to District 1
Total number of flips: 4
Flipping 34015020002 from District 2 to District 1
Total number of flips: 5
Flipping 34015025006 from District 2 to District 1
Total number of flips: 6
Flipping 34005065020 from District 3 to District 1
Total number of flips: 6
Flipping 34005065014 from District 3 to District 1
Total number of flips: 7
Flipping 34005040005 from District 3 to District 1
Total number of flips: 8
Flipping 34015025010 from District 2 to District 1
Total number of flips: 9
Flipping 34015005006 from District 2 to District 1
Total number of flips: 10
Flipping 34005065020 from District 3 to District 1
Total numb

In [21]:
fair_assignment = pd.read_csv('/content/New Jersey Data/precinct-assignments-congress-nj.csv')


print("New Jersey Fair Assignment Contiguity Check")
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(i, fair_assignment, nj_contiguity)))

New Jersey Fair Assignment Contiguity Check
District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True
